In [86]:
import pandas as pd
import string

data = [
    [1, 1, 1],
    [1, 2, 2],
    [2, 2, 1],
    [2, 3, 1],
    [3, 1, 1],
    [3, 2, 1],
    [3, 3, 1]
]

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

t = TfidfTransformer()
d = t.fit_transform(data)
print(d)


# spam_corpus = map(string., [ "buy viagra", "buy antibody" ])
# spam_corpus = ["buy", "cra", "buy", "antibody"]
# unique_words = set([word for doc in spam_corpus for word in doc])
# word_counts = [(word, map(lambda doc: doc.count(word), spam_corpus)) for word in unique_words]
# spam_bag_of_words = pd.DataFrame(dict(word_counts))
# print(spam_bag_of_words)

  (0, 2)	0.57735026919
  (0, 1)	0.57735026919
  (0, 0)	0.57735026919
  (1, 2)	0.666666666667
  (1, 1)	0.666666666667
  (1, 0)	0.333333333333
  (2, 2)	0.333333333333
  (2, 1)	0.666666666667
  (2, 0)	0.666666666667
  (3, 2)	0.267261241912
  (3, 1)	0.801783725737
  (3, 0)	0.534522483825
  (4, 2)	0.301511344578
  (4, 1)	0.301511344578
  (4, 0)	0.904534033733
  (5, 2)	0.267261241912
  (5, 1)	0.534522483825
  (5, 0)	0.801783725737
  (6, 2)	0.229415733871
  (6, 1)	0.688247201612
  (6, 0)	0.688247201612


In [87]:
measurements = [
   {'city': 'Dubai', 'temperature': 33.},
   {'city': 'Dubai', 'temperature': 3.},
   {'city': 'London', 'temperature': 12.},
   {'city': 'London', 'temperature': 1.},
   {'city': 'San Fransisco', 'temperature': 18.}]
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
vec.fit_transform(measurements).toarray()

array([[  1.,   0.,   0.,  33.],
       [  1.,   0.,   0.,   3.],
       [  0.,   1.,   0.,  12.],
       [  0.,   1.,   0.,   1.],
       [  0.,   0.,   1.,  18.]])

In [88]:
from sklearn.feature_extraction import DictVectorizer
import numpy as np

v = DictVectorizer(sparse=False)
D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
bag = [
    [1, 10, 10],
    [1, 20, 20],
    [2, 20, 10],
    [2, 30, 10],
    [3, 10, 10],
    [3, 20, 10],
    [3, 30, 10]
]
bagarray = np.asarray(bag)
# print(set(bagarray[...,0]))
bagdicts = []
for i in set(bagarray[...,0]):
    bagdicts.append({})
for w in bag:
    docid = w[0]
    wordid = w[1]
    wordcount = w[2]
    dict = bagdicts[docid-1]
    dict[wordid] = wordcount
# print(bagdicts)
    
# D2 = [
#     {1: 1, 2: 2},
#     {2: 1, 3: 1},
#     {1: 1, 2: 1, 3: 1}
# ]
X = v.fit_transform(bagdicts)
X
# v.inverse_transform(X) == \
#     [{'bar': 2.0, 'foo': 1.0}, {'baz': 1.0, 'foo': 3.0}]
# True
# >>> v.transform({'foo': 4, 'unseen_feature': 3})
# array([[ 0.,  0.,  4.]])

array([[ 10.,  20.,   0.],
       [  0.,  10.,  10.],
       [ 10.,  10.,  10.]])

In [89]:
vocabfile = "vocab.kos.txt"
with open(vocabfile, 'r') as f:
    vocab = [line.strip() for line in f.readlines()]
# print(vocab[0:10])

line = "1 61 2\n"
line.split()

docwordfile = "docword.kos.txt"
# bagdicts = [{} for i in range(0, len(vocab))]
bagdicts = []
count = 0
with open(docwordfile, 'r') as f:
    for line in f.readlines():
        w = line.strip().split()
        if len(w) == 3:
            docid = int(w[0]) - 1
            wordid = int(w[1]) - 1
            wordcount = int(w[2])
            if len(bagdicts) <= docid:
                bagdicts.append({})  # fuuuu
            dict = bagdicts[docid]
            # dict[vocab[wordid]] = wordcount
            dict[wordid] = wordcount
            count += 1
# print(count)
print(len(bagdicts), "files")
print(len(vocab), "terms in vocabulary")

sum = 0
for w in bagdicts[0]:
    sum += bagdicts[0].get(w)
print(sum, sum == 137)

3430 files
6906 terms in vocabulary
137 True


In [90]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=False)
counts = v.fit_transform(bagdicts)
sum = np.sum(counts[0])
print(sum, sum == 137)

137.0 True


In [91]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(counts)
print("m samples: %d, n features: %d" % tfidf.shape)
print()

m samples: 3430, n features: 6906



In [92]:
from time import time

from sklearn.cluster import KMeans
from sklearn import metrics

clusters = 3
km = KMeans(n_clusters=clusters,
            init='k-means++',  # or 'random' (random centroids) 
            n_init=10,  # number of time the k-means algorithm will be run with different centroid seeds.    
            max_iter=300
            )

print("Document clustering with %s" % km)
start = time()
km.fit(tfidf)
print("done in %0.3fs" % (time() - start))
print()

print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()
for i in range(clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % vocab[ind], end='')
    print()
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
# print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
# print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(labels, km.labels_))
# print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, km.labels_, sample_size=1000))
# print()

Document clustering with KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)


done in 2.066s

Top terms per cluster:
Cluster 0: november account electoral governor poll senate polls house republicans vote
Cluster 1: bush kerry iraq president war campaign general house poll administration
Cluster 2: dean edwards clark kerry lieberman primary gephardt poll iowa democratic


In [93]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
corpus = [
'This is the first document.',
'This is the second second document.',
'And the third one.',
'Is this the first document?',
]
X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

Transform a count matrix to a normalized tf or tf-idf representation

Tf means term-frequency while tf-idf means term-frequency times inverse
document-frequency. This is a common term weighting scheme in information
retrieval, that has also found good use in document classification.

The goal of using tf-idf instead of the raw frequencies of occurrence of a
token in a given document is to scale down the impact of tokens that occur
very frequently in a given corpus and that are hence empirically less
informative than features that occur in a small fraction of the training
corpus.

The actual formula used for tf-idf is tf * (idf + 1) = tf + tf * idf,
instead of tf * idf. The effect of this is that terms with zero idf, i.e.
that occur in all documents of a training set, will not be entirely
ignored. The formulas used to compute tf and idf depend on parameter
settings that correspond to the SMART notation used in IR, as follows:

Tf is "n" (natural) by default, "l" (logarithmic) when sublinear_tf=True.
Idf is "t" when use_idf is given, "n" (none) otherwise.
Normalization is "c" (cosine) when norm='l2', "n" (none) when norm=None.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Parameters
----------
norm : 'l1', 'l2' or None, optional
    Norm used to normalize term vectors. None for no normalization.

use_idf : boolean, default=True
    Enable inverse-document-frequency reweighting.

smooth_idf : boolean, default=True
    Smooth idf weights by adding one to document frequencies, as if an
    extra document was seen containing every term in the collection
    exactly once. Prevents zero divisions.

sublinear_tf : boolean, default=False
    Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).

References
----------

.. [Yates2011] `R. Baeza-Yates and B. Ribeiro-Neto (2011). Modern
               Information Retrieval. Addison Wesley, pp. 68-74.`

.. [MRS2008] `C.D. Manning, P. Raghavan and H. Schuetze  (2008).
               Introduction to Information Retrieval. Cambridge University
               Press, pp. 118-120.`

In [94]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()
counts = [
    [3, 0, 1], # doc1?
    [2, 0, 0], # doc2?
    [3, 0, 0],
    [4, 0, 0],
    [3, 2, 0],
    [3, 0, 2]]
tfidf = transformer.fit_transform(counts)
print(tfidf.toarray())
transformer.idf_

[[ 0.85151335  0.          0.52433293]
 [ 1.          0.          0.        ]
 [ 1.          0.          0.        ]
 [ 1.          0.          0.        ]
 [ 0.55422893  0.83236428  0.        ]
 [ 0.63035731  0.          0.77630514]]


array([ 1.        ,  2.25276297,  1.84729786])